#### Initial Setup

In [1]:
import os

In [2]:
%pwd

'E:\\RajaRajeshwari\\MyFolders\\Projects\\mlops_water_potability_prediction\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'E:\\RajaRajeshwari\\MyFolders\\Projects\\mlops_water_potability_prediction'

#### Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    status_file: Path

#### Configuration

In [6]:
from src.mlops_water_potability_prediction_project.constants import *
from src.mlops_water_potability_prediction_project.utilities.helpers import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories(directories_path_list=[self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            status_file=config.status_file
        )

        return data_transformation_config

#### Component

In [7]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from src.mlops_water_potability_prediction_project import logger


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.train = None
        self.test = None

    def split_dataset(self):
        try:
            dataframe = pd.read_csv(self.config.data_path)

            self.train, self.test = train_test_split(dataframe)

            logger.info("Split the dataset into train and test sets")
            logger.info(f"Train set shape: {self.train.shape}")
            logger.info(f"Test set shape: {self.test.shape}")

        except Exception as e:
            raise e

    def feature_scale_dataset(self):
        try:           
            sc = StandardScaler()

            train_array = self.train.to_numpy()
            test_array = self.test.to_numpy()
            
            train_array[:, :-1] = sc.fit_transform(train_array[:, :-1])
            test_array[:, :-1] = sc.transform(test_array[:, :-1])

            self.train = pd.DataFrame(train_array, columns=list(self.train.columns))
            self.test = pd.DataFrame(test_array, columns=list(self.test.columns))
            #print(self.train)
            #print(self.test)
                             
            logger.info("Feature scaled the train and test sets")
        except Exception as e:
            raise e

    def save_transformation(self):
        try:
            self.train.to_csv(os.path.join(self.config.root_dir, "train_set.csv"), index=False)
            self.test.to_csv(os.path.join(self.config.root_dir, "test_set.csv"), index=False)
                             
            logger.info("Saved the transformations as csv")
        except Exception as e:
            raise e


#### Pipeline

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.split_dataset()
    data_transformation.feature_scale_dataset()
    data_transformation.save_transformation()
except Exception as e:
    raise e

[2024-01-28 22:27:06,518]: INFO: helpers: YAML file: config\config.yaml loaded successfully]
[2024-01-28 22:27:06,522]: INFO: helpers: YAML file: params.yaml loaded successfully]
[2024-01-28 22:27:06,525]: INFO: helpers: YAML file: schema.yaml loaded successfully]
[2024-01-28 22:27:06,526]: INFO: helpers: Created directory at: artifacts]
[2024-01-28 22:27:06,527]: INFO: helpers: Created directory at: artifacts/data_transformation]
[2024-01-28 22:27:06,535]: INFO: 4115176146: Split the dataset into train and test sets]
[2024-01-28 22:27:06,536]: INFO: 4115176146: Train set shape: (1508, 10)]
[2024-01-28 22:27:06,537]: INFO: 4115176146: Test set shape: (503, 10)]
[2024-01-28 22:27:06,539]: INFO: 4115176146: Feature scaled the train and test sets]
[2024-01-28 22:27:06,562]: INFO: 4115176146: Saved the transformations as csv]
